In [1]:
%cd ..

/Users/viktoriaknazkova/Desktop/me/study/3 курс/инф_поиск/hw2-vknyazkova


In [2]:
!pip install -r requirements.txt

  Using cached https://github.com/explosion/spacy-models/releases/download/ru_core_news_sm-3.5.0/ru_core_news_sm-3.5.0-py3-none-any.whl (15.3 MB)


In [3]:
import time
import tracemalloc
import warnings
from collections import defaultdict

warnings.filterwarnings("ignore")

In [4]:
from thesis_search.cli.main import search
from thesis_search.search import search_theses

In [5]:
def time_decorator(f):
    def wrapper(*args, **kwds):
        start = time.time()
        f(*args, **kwds)
        exec_time = time.time() - start
        return exec_time
    return wrapper

In [6]:
# source: https://www.geeksforgeeks.org/monitoring-memory-usage-of-a-running-python-program/
def memory_decorator(f):
    def wrapper(*args, **kwds):
        tracemalloc.start()
        f(*args, **kwds)
        mem = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        return mem[1]
    return wrapper

In [7]:
exec_time = defaultdict(dict)
memory_usage = defaultdict(dict)
methods = ['bm25', 'w2v', 'ft', 'bert']

In [8]:
query = 'векторное представление многозначных слов'

### Поиск с загрузкой модели

In [9]:
search_time = time_decorator(search)
search_mem = memory_decorator(search)

In [10]:
%%capture
for method in methods:
    exec_time['full'][method] = search_time(query, method, 1)
    memory_usage['full'][method] = search_mem(query, method, 1)

In [11]:
exec_time, memory_usage

(defaultdict(dict,
             {'full': {'bm25': 0.213637113571167,
               'w2v': 0.9831628799438477,
               'ft': 9.398283004760742,
               'bert': 6.3626110553741455}}),
 defaultdict(dict,
             {'full': {'bm25': 629947561,
               'w2v': 5853920,
               'ft': 5851351,
               'bert': 19752993}}))

### Только поиск

In [12]:
from thesis_search import init_defaults

In [13]:
db, fast_nlp, corpus, default_params, search_engines = init_defaults()

In [14]:
search_engines_initialized = {}
for idx_type in methods:
    search_engine = search_engines[idx_type](corpus=corpus[idx_type], **default_params[idx_type])
    search_engines_initialized[idx_type] = search_engine

In [15]:
search_only_time = time_decorator(search_theses)
search_only_mem = memory_decorator(search_theses)

In [16]:
%%capture
for method in methods:
    exec_time['search_only'][method] = search_only_time(query, fast_nlp, search_engines_initialized[method], db, 1)
    memory_usage['search_only'][method] = search_only_mem(query, fast_nlp, search_engines_initialized[method], db, 1)

In [17]:
exec_time, memory_usage

(defaultdict(dict,
             {'full': {'bm25': 0.213637113571167,
               'w2v': 0.9831628799438477,
               'ft': 9.398283004760742,
               'bert': 6.3626110553741455},
              'search_only': {'bm25': 0.08329892158508301,
               'w2v': 0.03309798240661621,
               'ft': 0.023878097534179688,
               'bert': 0.523726224899292}}),
 defaultdict(dict,
             {'full': {'bm25': 629947561,
               'w2v': 5853920,
               'ft': 5851351,
               'bert': 19752993},
              'search_only': {'bm25': 191387,
               'w2v': 2973157,
               'ft': 2970711,
               'bert': 9921676}}))

In [19]:
from pathlib import Path
import pandas as pd

from thesis_search.config import DATA_FOLDER_PATH

In [20]:
time_info_path = Path(DATA_FOLDER_PATH, 'time_statistics.csv')
memory_info_path = Path(DATA_FOLDER_PATH, 'memory_statistics.csv')

In [22]:
pd.DataFrame.from_dict(exec_time).to_csv(time_info_path)
pd.DataFrame.from_dict(memory_usage).to_csv(memory_info_path)